# Import Required Libraries

In [1]:
import tensorflow.keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, Activation, AveragePooling3D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, MaxPooling3D,Conv3D
from tensorflow.keras.layers import Concatenate, SpatialDropout2D, SpatialDropout3D
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras import initializers
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D,ZeroPadding3D, BatchNormalization, Flatten,Conv3D,Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.utils import plot_model
import collections
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import backend as k
import numpy as np
import pandas as pd
import scipy.ndimage as ndi
import sklearn.metrics as metrics
import seaborn as sns

In [2]:
t2_tra_samples = np.load('C:/Sapna/Graham/Capstone/data/train/generated/numpy/tse_tra/X_train_t2_3D.npy',allow_pickle=True)
t2_tra_labels  = np.load('C:/Sapna/Graham/Capstone/data/train/generated/numpy/tse_tra/Y_train_t2_3D.npy',allow_pickle=True)
adc_samples    = np.load('C:/Sapna/Graham/Capstone/data/train/generated/numpy/adc/X_train_adc_3D.npy',allow_pickle=True)
bval_samples   = np.load('C:/Sapna/Graham/Capstone/data/train/generated/numpy/bval/X_train_bval_3D.npy',allow_pickle=True)

In [3]:
print(t2_tra_samples.shape)
print(bval_samples.shape)
print(adc_samples.shape)

(112, 3, 60, 60)
(112, 3, 16, 16)
(112, 3, 16, 16)


In [4]:
# factor = 64/t2_tra_samples.shape[2]
# print(factor)
# t2_tra_samples_v1 = ndi.zoom(t2_tra_samples, (1,1, factor, factor), order=2)
# print(t2_tra_samples_v1.shape)
      
# factor = 64/adc_samples.shape[2]
# print(factor)
# adc_samples_v1 = ndi.zoom(adc_samples, (1,1, factor, factor), order=2)
# print(adc_samples_v1.shape)
      
# factor = 64/bval_samples.shape[2]
# print(factor)
# bval_samples_v1 = ndi.zoom(bval_samples, (1,1, factor, factor), order=2)
# print(t2_tra_samples_v1.shape)

In [5]:
t2_samples_v1 = np.expand_dims(t2_tra_samples, axis=1)
adc_samples_v1 = np.expand_dims(adc_samples, axis=1)
bval_samples_v1 = np.expand_dims(bval_samples, axis=1)
print(t2_samples_v1.shape)
print(adc_samples_v1.shape)
print(bval_samples_v1.shape)

(112, 1, 3, 60, 60)
(112, 1, 3, 16, 16)
(112, 1, 3, 16, 16)


In [6]:
t2_samples_v2    = t2_samples_v1.reshape((112,60,60,3,1))
adc_samples_v2   = adc_samples_v1.reshape((112,16,16,3,1))
bval_samples_v2  = bval_samples_v1.reshape((112,16,16,3,1))

# print("#####################################################")
print(t2_samples_v2.shape)
print(adc_samples_v2.shape)
print(bval_samples_v2.shape)

(112, 60, 60, 3, 1)
(112, 16, 16, 3, 1)
(112, 16, 16, 3, 1)


In [7]:
target_tra = []
for i in t2_tra_labels:
    if i == 1:
        target_tra.append("0")
    elif i == 2:
        target_tra.append("1")
    elif i == 3:
        target_tra.append("2")
    elif i == 4:
        target_tra.append("3")
    else:
        target_tra.append("4")

print("The frequency count of the GS:")
collections.Counter(target_tra)

The frequency count of the GS:


Counter({'2': 20, '0': 36, '1': 41, '3': 8, '4': 7})

In [8]:
t2_labels = np.array(pd.get_dummies(target_tra))

In [9]:
t2_categorical = pd.DataFrame(t2_labels).idxmax(axis=1)
print(collections.Counter(t2_categorical))

Counter({1: 41, 0: 36, 2: 20, 3: 8, 4: 7})


# Split the train and test data

In [10]:
x_train_t2,x_test_t2,y_train_t2,y_test_t2 = train_test_split(t2_samples_v2,t2_labels,
                                                                     test_size=0.20, 
                                                                     stratify=t2_labels,  
                                                                     random_state=42)

x_train_t2,x_val_t2,y_train_t2,y_val_t2 = train_test_split(x_train_t2,y_train_t2,
                                                                     test_size=0.20, 
                                                                     stratify=y_train_t2,  
                                                                     random_state=42)

categorical_train_labels = pd.DataFrame(y_train_t2).idxmax(axis=1)
print(collections.Counter(categorical_train_labels))
categorical_val_labels = pd.DataFrame(y_val_t2).idxmax(axis=1)
print(collections.Counter(categorical_val_labels))
categorical_test_labels = pd.DataFrame(y_test_t2).idxmax(axis=1)
print(collections.Counter(categorical_test_labels))
print(x_train_t2.shape)
print(x_test_t2.shape)

Counter({1: 26, 0: 23, 2: 13, 3: 5, 4: 4})
Counter({1: 7, 0: 6, 2: 3, 4: 1, 3: 1})
Counter({1: 8, 0: 7, 2: 4, 3: 2, 4: 2})
(71, 60, 60, 3, 1)
(23, 60, 60, 3, 1)


In [11]:
x_train_adc,x_test_adc,y_train_adc,y_test_adc = train_test_split(adc_samples_v2,t2_labels,
                                                                     test_size=0.20, 
                                                                     stratify=t2_labels,  
                                                                     random_state=42)

x_train_adc,x_val_adc,y_train_adc,y_val_adc = train_test_split(x_train_adc,y_train_adc,
                                                                     test_size=0.20, 
                                                                     stratify=y_train_adc,  
                                                                     random_state=42)

categorical_train_labels = pd.DataFrame(y_train_adc).idxmax(axis=1)
print(collections.Counter(categorical_train_labels))
categorical_val_labels = pd.DataFrame(y_val_adc).idxmax(axis=1)
print(collections.Counter(categorical_val_labels))
categorical_test_labels = pd.DataFrame(y_test_adc).idxmax(axis=1)
print(collections.Counter(categorical_test_labels))
print(x_train_adc.shape)
print(x_test_adc.shape)

Counter({1: 26, 0: 23, 2: 13, 3: 5, 4: 4})
Counter({1: 7, 0: 6, 2: 3, 4: 1, 3: 1})
Counter({1: 8, 0: 7, 2: 4, 3: 2, 4: 2})
(71, 16, 16, 3, 1)
(23, 16, 16, 3, 1)


In [12]:
x_train_bval,x_test_bval,y_train_bval,y_test_bval = train_test_split(bval_samples_v2,t2_labels,
                                                                     test_size=0.20, 
                                                                     stratify=t2_labels,  
                                                                     random_state=42)

x_train_bval,x_val_bval,y_train_bval,y_val_bval = train_test_split(x_train_bval,y_train_bval,
                                                                     test_size=0.20, 
                                                                     stratify=y_train_bval,  
                                                                     random_state=42)

categorical_train_labels = pd.DataFrame(y_train_bval).idxmax(axis=1)
print(collections.Counter(categorical_train_labels))
categorical_val_labels = pd.DataFrame(y_val_bval).idxmax(axis=1)
print(collections.Counter(categorical_val_labels))
categorical_test_labels = pd.DataFrame(y_test_bval).idxmax(axis=1)
print(collections.Counter(categorical_test_labels))
print(x_train_bval.shape)
print(x_test_bval.shape)

Counter({1: 26, 0: 23, 2: 13, 3: 5, 4: 4})
Counter({1: 7, 0: 6, 2: 3, 4: 1, 3: 1})
Counter({1: 8, 0: 7, 2: 4, 3: 2, 4: 2})
(71, 16, 16, 3, 1)
(23, 16, 16, 3, 1)


In [13]:
t2_img_height, t2_img_width, t2_img_depth, t2_channel = x_train_t2.shape[1],x_train_t2.shape[2],x_train_t2.shape[3],x_train_t2.shape[4]
adc_img_height, adc_img_width, adc_img_depth,adc_channel = x_train_adc.shape[1],x_train_adc.shape[2],x_train_adc.shape[3],x_train_adc.shape[4]
bval_img_height, bval_img_width,bval_img_depth, bval_channel = x_train_bval.shape[1],x_train_bval.shape[2],x_train_bval.shape[3],x_train_bval.shape[4]

print(t2_img_height)
print(t2_img_width)
print(t2_img_depth)
print(t2_channel)

# print(adc_img_height)
# print(adc_img_width)
# print(adc_img_depth)
# print(adc_channel)

# print(bval_img_height)
# print(bval_img_width)
# print(bval_img_depth)
# print(bval_channel)

60
60
3
1


In [14]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_D_prev ,n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_D, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res_t2' + str(stage) + block + '_branch'
    bn_name_base = 'bn_t2' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv3D(filters = F1, kernel_size = (1,1,1), strides = (1,1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
 
    # Second component of main path (≈3 lines)
    X = Conv3D(filters = F2, kernel_size = (f,f,f), strides = (1,1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv3D(filters = F3, kernel_size = (1,1,1), strides = (1,1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev,n_D_prev , n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_D,n_C)
    """
    
    # defining name basis
    conv_name_base = 'res_t2' + str(stage) + block + '_branch'
    bn_name_base = 'bn_t2' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv3D(F1, (1, 1,1), strides = (s,s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv3D(filters = F2, kernel_size = (f,f,f), strides = (1,1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)


    # Third component of main path (≈2 lines)
    X = Conv3D(filters = F3, kernel_size = (1,1,1), strides = (1,1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv3D(filters = F3, kernel_size = (1,1,1), strides = (s,s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=42))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X


def ResNet50(input_shape = (t2_img_height, t2_img_width, t2_img_depth, t2_channel)):
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding3D((3, 3,3))(X_input)
    
    # Stage 1
    X = Conv3D(64, (7, 7,7), strides = (2, 2,2), name = 'conv1_t2', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1_t2')(X)
    X = Activation('relu')(X)
    X = MaxPooling3D((3, 3,2), strides=(2, 2,2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = AveragePooling3D(pool_size=(2,2,2), padding='same')(X)

    # Output layer
    X = Flatten()(X)
    X = Dense(32, activation='relu', name='fc_t2', kernel_initializer = glorot_uniform(seed=42))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50_t2')

    return model

t2_model = ResNet50(input_shape = (t2_img_height, t2_img_width, t2_img_depth,t2_channel))
t2_model.summary()

Model: "ResNet50_t2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 60, 60, 3, 1 0                                            
__________________________________________________________________________________________________
zero_padding3d (ZeroPadding3D)  (None, 66, 66, 9, 1) 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_t2 (Conv3D)               (None, 30, 30, 2, 64 22016       zero_padding3d[0][0]             
__________________________________________________________________________________________________
bn_conv1_t2 (BatchNormalization (None, 30, 30, 2, 64 8           conv1_t2[0][0]                   
________________________________________________________________________________________

In [15]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_D_prev ,n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_D, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res_adc' + str(stage) + block + '_branch'
    bn_name_base = 'bn_adc' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv3D(filters = F1, kernel_size = (1,1,1), strides = (1,1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
 
    # Second component of main path (≈3 lines)
    X = Conv3D(filters = F2, kernel_size = (f,f,f), strides = (1,1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv3D(filters = F3, kernel_size = (1,1,1), strides = (1,1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev,n_D_prev , n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_D,n_C)
    """
    
    # defining name basis
    conv_name_base = 'res_adc' + str(stage) + block + '_branch'
    bn_name_base = 'bn_adc' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv3D(F1, (1, 1,1), strides = (s,s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv3D(filters = F2, kernel_size = (f,f,f), strides = (1,1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)


    # Third component of main path (≈2 lines)
    X = Conv3D(filters = F3, kernel_size = (1,1,1), strides = (1,1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv3D(filters = F3, kernel_size = (1,1,1), strides = (s,s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=42))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X


def ResNet50(input_shape = (adc_img_height, adc_img_width, adc_img_depth, adc_channel)):
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding3D((3, 3,3))(X_input)
    
    # Stage 1
    X = Conv3D(64, (7, 7,7), strides = (2, 2,2), name = 'conv1_adc', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1_adc')(X)
    X = Activation('relu')(X)
    X = MaxPooling3D((3, 3,2), strides=(2, 2,2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = AveragePooling3D(pool_size=(2,2,2), padding='same')(X)

    # Output layer
    X = Flatten()(X)
    X = Dense(32, activation='relu', name='fc_adc', kernel_initializer = glorot_uniform(seed=42))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50_adc')

    return model

adc_model = ResNet50(input_shape = (adc_img_height, adc_img_width, adc_img_depth,adc_channel))
adc_model.summary()

Model: "ResNet50_adc"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 16, 16, 3, 1 0                                            
__________________________________________________________________________________________________
zero_padding3d_1 (ZeroPadding3D (None, 22, 22, 9, 1) 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_adc (Conv3D)              (None, 8, 8, 2, 64)  22016       zero_padding3d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1_adc (BatchNormalizatio (None, 8, 8, 2, 64)  8           conv1_adc[0][0]                  
_______________________________________________________________________________________

In [16]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_D_prev ,n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_D, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res_bval' + str(stage) + block + '_branch'
    bn_name_base = 'bn_bval' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv3D(filters = F1, kernel_size = (1,1,1), strides = (1,1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
 
    # Second component of main path (≈3 lines)
    X = Conv3D(filters = F2, kernel_size = (f,f,f), strides = (1,1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv3D(filters = F3, kernel_size = (1,1,1), strides = (1,1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev,n_D_prev , n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_D,n_C)
    """
    
    # defining name basis
    conv_name_base = 'res_bval' + str(stage) + block + '_branch'
    bn_name_base = 'bn_bval' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv3D(F1, (1, 1,1), strides = (s,s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv3D(filters = F2, kernel_size = (f,f,f), strides = (1,1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)


    # Third component of main path (≈2 lines)
    X = Conv3D(filters = F3, kernel_size = (1,1,1), strides = (1,1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv3D(filters = F3, kernel_size = (1,1,1), strides = (s,s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=42))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X


def ResNet50(input_shape = (bval_img_height, bval_img_width, bval_img_depth,bval_channel)):
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding3D((3, 3,3))(X_input)
    
    # Stage 1
    X = Conv3D(64, (7, 7,7), strides = (2, 2,2), name = 'conv1_bval', kernel_initializer = glorot_uniform(seed=42))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1_bval')(X)
    X = Activation('relu')(X)
    X = MaxPooling3D((3, 3,2), strides=(2, 2,2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = AveragePooling3D(pool_size=(2,2,2), padding='same')(X)

    # Output layer
    X = Flatten()(X)
    X = Dense(32, activation='relu', name='fc_bval', kernel_initializer = glorot_uniform(seed=42))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50_bval')

    return model

bval_model = ResNet50(input_shape = (bval_img_height, bval_img_width, bval_img_depth, bval_channel))
bval_model.summary()

Model: "ResNet50_bval"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 16, 16, 3, 1 0                                            
__________________________________________________________________________________________________
zero_padding3d_2 (ZeroPadding3D (None, 22, 22, 9, 1) 0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_bval (Conv3D)             (None, 8, 8, 2, 64)  22016       zero_padding3d_2[0][0]           
__________________________________________________________________________________________________
bn_conv1_bval (BatchNormalizati (None, 8, 8, 2, 64)  8           conv1_bval[0][0]                 
______________________________________________________________________________________

In [17]:
classes=5
mergedOutput = Concatenate()([t2_model.output, adc_model.output, bval_model.output])
# # mergedOutput = Concatenate()([t2_model.output, adc_model.output])
# # out = Dense(256, activation='relu')(mergedOutput)
# # out = Dense(128, activation='relu')(out)
# # out = Dense(64, activation='relu')(out)
# out = Dropout(0.8)(out)
# # out = Dense(32, activation='relu')(out)
out = Dense(16, activation='relu')(mergedOutput)
out = Dense(classes, activation='softmax')(out)

In [18]:
full_model = Model([t2_model.input, adc_model.input, bval_model.input],out) 
# # full_model = Model([t2_model.input, adc_model.input],out) 
# out = Dense(classes, activation='softmax')(t2_model.output)
# full_model = Model([t2_model.input],out) 
# full_model.summary()

In [19]:
plot_model(full_model,  show_shapes=True)
# full_model.summary()

In [20]:
full_model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [21]:
checkpoint_vl = ModelCheckpoint("weights.best_vl.hdf5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
checkpoint_va = ModelCheckpoint("weights.best_va.hdf5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint_va, checkpoint_vl]
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

In [22]:
x_train_t2_v1 = x_train_t2/255.
x_val_t2_v1   = x_val_t2/255.
x_test_t2_v1  = x_test_t2/255.

y_train_t2_v1 = y_train_t2
y_val_t2_v1   = y_val_t2
y_test_t2_v1  = y_test_t2

x_train_adc_v1 = x_train_adc/255.
x_val_adc_v1   = x_val_adc/255.
x_test_adc_v1  = x_test_adc/255.

y_train_adc_v1 = y_train_adc
y_val_adc_v1   = y_val_adc
y_test_adc_v1  = y_test_adc

x_train_bval_v1 = x_train_bval/255.
x_val_bval_v1   = x_val_bval/255.
x_test_bval_v1  = x_test_bval/255.

y_train_bval_v1 = y_train_bval
y_val_bval_v1   = y_val_bval
y_test_bval_v1  = y_test_bval

In [ ]:
history = full_model.fit([x_train_t2_v1,x_train_adc_v1,x_train_bval_v1], 
#     x_train_t2_v1,
#     [x_train_t2_v1,x_train_adc_v1],    
                            y = y_train_t2_v1, 
                            batch_size = 64, 
                            epochs = 200, 
                            verbose = 1, 
                            validation_data =([x_val_t2_v1,x_val_adc_v1,x_val_bval_v1], y_val_t2_v1), 
#                          validation_data =([x_val_t2_v1,x_val_adc_v1], y_val_t2_v1), 
#                          validation_data =([x_val_t2_v1], y_val_t2_v1), 
                            shuffle = True, 
                            callbacks = [es])

Epoch 1/200
2/2 [==============================] - 15s 8s/step - loss: 3.1555 - accuracy: 0.2113 - val_loss: 29.1201 - val_accuracy: 0.3889
Epoch 2/200
2/2 [==============================] - 4s 2s/step - loss: 13.3224 - accuracy: 0.3662 - val_loss: 6708.3091 - val_accuracy: 0.3889
Epoch 3/200
2/2 [==============================] - 4s 2s/step - loss: 3.3096 - accuracy: 0.2254 - val_loss: 324018.1875 - val_accuracy: 0.3889
Epoch 4/200
2/2 [==============================] - 4s 2s/step - loss: 1.7603 - accuracy: 0.3803 - val_loss: 2962944.0000 - val_accuracy: 0.3889
Epoch 5/200
2/2 [==============================] - 4s 2s/step - loss: 1.7439 - accuracy: 0.2676 - val_loss: 11653368.0000 - val_accuracy: 0.3889
Epoch 6/200
2/2 [==============================] - 4s 2s/step - loss: 1.5647 - accuracy: 0.3239 - val_loss: 21043192.0000 - val_accuracy: 0.3889
Epoch 7/200
2/2 [==============================] - 5s 2s/step - loss: 1.5105 - accuracy: 0.2958 - val_loss: 22912552.0000 - val_accuracy: 0.3

In [ ]:
# preds = full_model.evaluate(x=[x_test_t2_v1,x_test_adc_v1,x_test_bval_v1],y=y_test_t2_v1)
preds = full_model.evaluate(x=[x_test_t2_v1],y=y_test_t2_v1)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
# full_model.save_weights('res_3D_v2.h5')

In [ ]:
def plot_metrics(history, sequence_type):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc)+1)

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title(sequence_type + ' training and validation accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    acc_v1 = round(acc[-1]*100 ,0)
    plt.figure()
    
#     print("Model Accuracy:", acc_v1 )
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title(sequence_type + ' training and validation loss')
    plt.xlabel('Epoch')
    plt.legend()
    
    plt.show()
    return

In [ ]:
import matplotlib.pyplot as plt
plot_metrics(history, 'T2/ADC/BVAL')

In [ ]:
#Train 
predicted_train = full_model.predict([x_train_t2_v1,x_train_adc_v1])
predicted_train_pd = pd.DataFrame(predicted_train)

predicted_train_pd['Max'] = predicted_train_pd.idxmax(axis=1)
print(predicted_train_pd)

actual_train = pd.DataFrame(y_train_t2_v1).idxmax(axis=1)
print(collections.Counter(actual_train))

In [ ]:
# predicted = full_model.predict([x_test_t2_v1,x_test_adc_v1,x_test_bval_v1])
predicted_test = full_model.predict([x_test_t2_v1,x_test_adc_v1])
predicted_test_pd = pd.DataFrame(predicted_test)

predicted_test_pd['Max'] = predicted_test_pd.idxmax(axis=1)
predicted_test_pd

actual_test = pd.DataFrame(y_test_t2_v1).idxmax(axis=1)
print(collections.Counter(actual_test))

In [ ]:
from sklearn.metrics import cohen_kappa_score , accuracy_score
print("Train Accuracy", accuracy_score(actual_train,predicted_train_pd['Max']))
print("Test Accuracy", accuracy_score(actual_test,predicted_test_pd['Max']))

# from sklearn.metrics import cohen_kappa_score , accuracy_score
# print("Kappa Score",cohen_kappa_score(actual_train,predicted_train_pd['Max']))

In [ ]:
# Train Confusion Matrix
cm = metrics.confusion_matrix(y_true=actual_train, y_pred=predicted_train_pd["Max"])
plt.figure(figsize=(9,9)) 
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap=None)
plt.ylabel('Actual label') 
plt.xlabel('Predicted label')

In [ ]:
print("Train Report",metrics.classification_report(actual_train, predicted_train_pd["Max"], labels=[0,1,2,3,4]))

In [ ]:
# Test Confusion Matrix
cm = metrics.confusion_matrix(y_true=actual_test, y_pred=predicted_test_pd["Max"])
plt.figure(figsize=(9,9)) 
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap=None)
plt.ylabel('Actual label') 
plt.xlabel('Predicted label')

In [ ]:
print("Test Report",metrics.classification_report(actual_test, predicted_test_pd["Max"], labels=[0,1,2,3,4]))